Lambda School Data Science

*Unit 2, Sprint 3, Module 2*

---


# Wrangle ML datasets 🍌


In today's lesson, we’ll work with a dataset of [3 Million Instacart Orders, Open Sourced](https://tech.instacart.com/3-million-instacart-orders-open-sourced-d40d29ead6f2)!



### Setup

In [1]:
# Download data
import requests

def download(url):
    filename = url.split('/')[-1]
    print(f'Downloading {url}')
    r = requests.get(url)
    with open(filename, 'wb') as f:
        f.write(r.content)
    print(f'Downloaded {filename}')

download('https://s3.amazonaws.com/instacart-datasets/instacart_online_grocery_shopping_2017_05_01.tar.gz')

Downloaded instacart_online_grocery_shopping_2017_05_01.tar.gz


In [2]:
# Uncompress data
import tarfile
tarfile.open('instacart_online_grocery_shopping_2017_05_01.tar.gz').extractall()

In [3]:
# Change directory to where the data was uncompressed
%cd instacart_2017_05_01

D:\Users\Jonathan\Documents\GitHub\DS-Unit-2-Applied-Modeling\module2\instacart_2017_05_01


In [4]:
# Print the csv filenames
from glob import glob
for filename in glob('*.csv'):
    print(filename)

aisles.csv
departments.csv
orders.csv
order_products__prior.csv
order_products__train.csv
products.csv


### For each csv file, look at its shape & head 

In [5]:
import pandas as pd
from IPython.display import display

def preview():
    for filename in glob('*.csv'):
        df = pd.read_csv(filename)
        print(filename, df.shape)
        display(df.head())
        print('\n')

preview()

aisles.csv (134, 2)


,aisle_id,aisle
0,1,prepared soups salads
1,2,specialty cheeses
2,3,energy granola bars
3,4,instant foods
4,5,marinades meat preparation




departments.csv (21, 2)


,department_id,department
0,1,frozen
1,2,other
2,3,bakery
3,4,produce
4,5,alcohol




orders.csv (3421083, 7)


...

,order_id,product_id,add_to_cart_order,reordered
0,2,33120,1,1
1,2,28985,2,1
2,2,9327,3,0
3,2,45918,4,1
4,2,30035,5,0




order_products__train.csv (1384617, 4)


,order_id,product_id,add_to_cart_order,reordered
0,1,49302,1,1
1,1,11109,2,1
2,1,10246,3,0
3,1,49683,4,0
4,1,43633,5,1




products.csv (49688, 4)


,product_id,product_name,aisle_id,department_id
0,1,Chocolate Sandwich Cookies,61,19
1,2,All-Seasons Salt,104,13
2,3,Robust Golden Unsweetened Oolong Tea,94,7
3,4,Smart Ones Classic Favorites Mini Rigatoni Wit...,38,1
4,5,Green Chile Anytime Sauce,5,13


## The original task was complex ...

[The Kaggle competition said,](https://www.kaggle.com/c/instacart-market-basket-analysis/data):

> The dataset for this competition is a relational set of files describing customers' orders over time. The goal of the competition is to predict which products will be in a user's next order.

> orders.csv: This file tells to which set (prior, train, test) an order belongs. You are predicting reordered items only for the test set orders.

Each row in the submission is an order_id from the test set, followed by product_id(s) predicted to be reordered.

> sample_submission.csv: 
```
order_id,products
17,39276 29259
34,39276 29259
137,39276 29259
182,39276 29259
257,39276 29259
```

## ... but we can simplify!

Simplify the question, from "Which products will be reordered?" (Multi-class, [multi-label](https://en.wikipedia.org/wiki/Multi-label_classification) classification) to **"Will customers reorder this one product?"** (Binary classification)

Which product? How about **the most frequently ordered product?**

# Questions:

- What is the most frequently ordered product?
- How often is this product included in a customer's next order?
- Which customers have ordered this product before?
- How can we get a subset of data, just for these customers?
- What features can we engineer? We want to predict, will these customers reorder this product on their next order?

## What was the most frequently ordered product?

In [6]:
prior = pd.read_csv('order_products__prior.csv')

In [7]:
prior['product_id'].mode()

0    24852
dtype: int64

In [8]:
prior['product_id'].value_counts()

24852    472565
13176    379450
21137    264683
21903    241921
47209    213584
          ...  
11356         1
18001         1
6320          1
26268         1
30087         1
Name: product_id, Length: 49677, dtype: int64

In [9]:
train = pd.read_csv('order_products__train.csv')

In [10]:
train['product_id'].mode()

0    24852
dtype: int64

In [11]:
train['product_id'].value_counts()

24852    18726
13176    15480
21137    10894
21903     9784
47626     8135
         ...  
44256        1
2764         1
4815         1
43736        1
46835        1
Name: product_id, Length: 39123, dtype: int64

In [12]:
products = pd.read_csv('products.csv')

In [13]:
products[products['product_id']==24852]

,product_id,product_name,aisle_id,department_id
24851,24852,Banana,24,4


In [14]:
prior = pd.merge(prior, products, on='product_id')

## How often is this product included in a customer's next order?

There are [three sets of data](https://gist.github.com/jeremystan/c3b39d947d9b88b3ccff3147dbcf6c6b):

> "prior": orders prior to that users most recent order (3.2m orders)  
"train": training data supplied to participants (131k orders)  
"test": test data reserved for machine learning competitions (75k orders)

Customers' next orders are in the "train" and "test" sets. (The "prior" set has the orders prior to the most recent orders.)

We can't use the "test" set here, because we don't have its labels (only Kaggle & Instacart have them), so we don't know what products were bought in the "test" set orders.

So, we'll use the "train" set. It currently has one row per product_id and multiple rows per order_id.

But we don't want that. Instead we want one row per order_id, with a binary column: "Did the order include the product?"

Let's wrangle!

### Technique #1

In [16]:
df = train.head(16).copy()

In [25]:
df['bananas'] = df['product_id'] == 24852

In [27]:
df.groupby('order_id')['bananas'].any()

order_id
1     False
36    False
Name: bananas, dtype: bool

In [30]:
train['bananas'] = train['product_id'] == 24852
train.groupby('order_id')['bananas'].any().reset_index()

,order_id,bananas
0,1,False
1,36,False
2,38,False
3,96,False
4,98,False
...,...,...
131204,3421049,False
131205,3421056,False
131206,3421058,False
131207,3421063,False


In [31]:
train_wrangled = train.groupby('order_id')['bananas'].any().reset_index()

In [32]:
target = 'bananas'
train_wrangled[target].value_counts(normalize=True)

False    0.857281
True     0.142719
Name: bananas, dtype: float64

### Technique #2

In [33]:
df

,order_id,product_id,add_to_cart_order,reordered,bananas
0,1,49302,1,1,False
1,1,11109,2,1,False
2,1,10246,3,0,False
3,1,49683,4,0,False
4,1,43633,5,1,False
5,1,13176,6,0,False
6,1,47209,7,0,False
7,1,22035,8,1,False
8,36,39612,1,0,False
9,36,19660,2,1,False


In [34]:
# Group by order_id, get a list of product_ids for that order
df.groupby('order_id')['product_id'].apply(list)

order_id
1     [49302, 11109, 10246, 49683, 43633, 13176, 472...
36    [39612, 19660, 49235, 43086, 46620, 34497, 486...
Name: product_id, dtype: object

In [35]:
# Group by order_id, get a list of product_ids for that order
# Check if that list includes bananas

def includes_bananas(product_ids):
    return 24852 in list(product_ids)

df.groupby('order_id')['product_id'].apply(includes_bananas)

order_id
1     False
36    False
Name: product_id, dtype: bool

In [36]:
train = (train
         .groupby('order_id')
         .agg({'product_id': includes_bananas})
         .reset_index()
         .rename(columns={'product_id': 'bananas'}))

In [37]:
target = 'bananas'
train[target].value_counts(normalize=True)

False    0.857281
True     0.142719
Name: bananas, dtype: float64

## Which customers have ordered this product before?

- Customers are identified by `user_id`
- Products are identified by `product_id`

Do we have a table with both these id's? (If not, how can we combine this information?)

In [38]:
preview()

aisles.csv (134, 2)


,aisle_id,aisle
0,1,prepared soups salads
1,2,specialty cheeses
2,3,energy granola bars
3,4,instant foods
4,5,marinades meat preparation




departments.csv (21, 2)


,department_id,department
0,1,frozen
1,2,other
2,3,bakery
3,4,produce
4,5,alcohol




orders.csv (3421083, 7)


...

,order_id,product_id,add_to_cart_order,reordered
0,2,33120,1,1
1,2,28985,2,1
2,2,9327,3,0
3,2,45918,4,1
4,2,30035,5,0




order_products__train.csv (1384617, 4)


,order_id,product_id,add_to_cart_order,reordered
0,1,49302,1,1
1,1,11109,2,1
2,1,10246,3,0
3,1,49683,4,0
4,1,43633,5,1




products.csv (49688, 4)


,product_id,product_name,aisle_id,department_id
0,1,Chocolate Sandwich Cookies,61,19
1,2,All-Seasons Salt,104,13
2,3,Robust Golden Unsweetened Oolong Tea,94,7
3,4,Smart Ones Classic Favorites Mini Rigatoni Wit...,38,1
4,5,Green Chile Anytime Sauce,5,13


Answer:

No, we don't have a table with both these id's. But:
- `orders.csv` has `user_id` and `order_id`
- `order_products__prior.csv` has `order_id` and `product_id`
- `order_products__train.csv` has `order_id` and `product_id`, too

In [39]:
# In the order_products__prior table, which orders included bananas?

In [40]:
BANANAS = 24852
prior[prior.product_id==BANANAS]

,order_id,product_id,add_to_cart_order,reordered,product_name,aisle_id,department_id
1771449,10,24852,1,1,Banana,24,4
1771450,20,24852,6,0,Banana,24,4
1771451,22,24852,3,1,Banana,24,4
1771452,26,24852,2,1,Banana,24,4
1771453,52,24852,2,1,Banana,24,4
...,...,...,...,...,...,...,...
2244009,3421027,24852,3,1,Banana,24,4
2244010,3421030,24852,9,1,Banana,24,4
2244011,3421038,24852,2,0,Banana,24,4
2244012,3421078,24852,2,1,Banana,24,4


In [42]:
banana_prior_order_ids = prior[prior.product_id==BANANAS].order_id

In [43]:
# Look at the orders table
orders = pd.read_csv('orders.csv')
orders.sample(n=5)

,order_id,user_id,eval_set,order_number,order_dow,order_hour_of_day,days_since_prior_order
3103498,1292425,187151,prior,1,6,16,NaN
2447919,1222767,147311,prior,13,5,13,13.0
3007007,2498117,181427,train,4,1,5,2.0
586550,2541852,35345,prior,61,1,8,10.0
1750874,861648,105105,prior,15,5,20,8.0


In [44]:
# In the orders table, which orders included bananas?
orders[orders.order_id.isin(banana_prior_order_ids)]

,order_id,user_id,eval_set,order_number,order_dow,order_hour_of_day,days_since_prior_order
14,738281,2,prior,4,2,10,8.0
16,1199898,2,prior,6,2,9,13.0
17,3194192,2,prior,7,2,12,14.0
18,788338,2,prior,8,1,15,27.0
19,1718559,2,prior,9,2,9,8.0
...,...,...,...,...,...,...,...
3420915,1764570,206202,prior,20,4,0,11.0
3421078,2266710,206209,prior,10,5,18,29.0
3421079,1854736,206209,prior,11,4,10,30.0
3421080,626363,206209,prior,12,1,12,18.0


In [45]:
# Check this order id, confirm that it includes bananas
prior[prior.order_id==738281]

,order_id,product_id,add_to_cart_order,reordered,product_name,aisle_id,department_id
1313559,738281,47209,9,0,Organic Hass Avocado,24,4
1873670,738281,24852,8,0,Banana,24,4
6583903,738281,37646,6,0,Organic Gala Apples,24,4
11211287,738281,36735,5,0,Organic Roasted Sliced Chicken Breast,96,20
14468499,738281,45613,11,0,Pad Thai,38,1
15253763,738281,34688,4,0,Good Belly Probiotics Juice Drink Blueberry Ac...,31,7
18855616,738281,32139,3,0,Hommus Classic Original,67,20
22264522,738281,21150,13,0,Fire Grilled Steak Bowl,38,1
23977423,738281,22829,7,0,Multigrain Flax Seeded Flatbread Crackers,78,19
24989071,738281,32792,2,1,Chipotle Beef & Pork Realstick,23,19


In [46]:
banana_orders = orders[orders.order_id.isin(banana_prior_order_ids)]

In [49]:
# In the orders table, which users have bought bananas?
banana_user_ids = banana_orders.user_id.unique()

## How can we get a subset of data, just for these customers?

We want *all* the orders from customers who have *ever* bought bananas.

(And *none* of the orders from customers who have *never* bought bananas.)

In [50]:
# orders table, shape before getting subset
orders.shape

(3421083, 7)

In [52]:
# orders table, shape after getting subset
orders = orders[orders.user_id.isin(banana_user_ids)]
orders.shape

(1512975, 7)

In [53]:
# IDs of *all* the orders from customers who have *ever* bought bananas
subset_order_ids = orders.order_id.unique()

In [54]:
# order_products__prior table, shape before getting subset
prior.shape

(32434489, 7)

In [55]:
# order_products__prior table, shape after getting subset
prior = prior[prior.order_id.isin(subset_order_ids)]
prior.shape

(16534534, 7)

In [56]:
# order_products__train table, shape before getting subset
train.shape

(131209, 2)

In [58]:
# order_products__train table, shape after getting subset
train = train[train.order_id.isin(subset_order_ids)]
train.shape

(46964, 2)

In [59]:
# In the subset, how often were bananas reordered in the customer's most recent order?
train[target].value_counts(normalize=True)

False    0.647453
True     0.352547
Name: bananas, dtype: float64

## What features can we engineer? We want to predict, will these customers reorder this product on their next order?

- Other fruit they buy
- Time between banana orders
- Frequency of banana orders by a customer
- Organic or not
- Time of day

In [60]:
preview()

aisles.csv (134, 2)


,aisle_id,aisle
0,1,prepared soups salads
1,2,specialty cheeses
2,3,energy granola bars
3,4,instant foods
4,5,marinades meat preparation




departments.csv (21, 2)


,department_id,department
0,1,frozen
1,2,other
2,3,bakery
3,4,produce
4,5,alcohol




orders.csv (3421083, 7)


...

,order_id,product_id,add_to_cart_order,reordered
0,2,33120,1,1
1,2,28985,2,1
2,2,9327,3,0
3,2,45918,4,1
4,2,30035,5,0




order_products__train.csv (1384617, 4)


,order_id,product_id,add_to_cart_order,reordered
0,1,49302,1,1
1,1,11109,2,1
2,1,10246,3,0
3,1,49683,4,0
4,1,43633,5,1




products.csv (49688, 4)


,product_id,product_name,aisle_id,department_id
0,1,Chocolate Sandwich Cookies,61,19
1,2,All-Seasons Salt,104,13
2,3,Robust Golden Unsweetened Oolong Tea,94,7
3,4,Smart Ones Classic Favorites Mini Rigatoni Wit...,38,1
4,5,Green Chile Anytime Sauce,5,13


In [61]:
train.shape

(46964, 2)

In [62]:
train.head()

,order_id,bananas
0,1,False
1,36,False
9,349,False
13,631,False
18,878,True


In [63]:
# Merge user_id, order_number, order_dow, order_hour_of_day, and
# days_since_prior_order with the training data
train = pd.merge(train, orders)

In [64]:
train.head()

,order_id,bananas,user_id,eval_set,order_number,order_dow,order_hour_of_day,days_since_prior_order
0,1,False,112108,train,4,4,10,9.0
1,36,False,79431,train,23,6,18,30.0
2,349,False,156353,train,9,3,16,30.0
3,631,False,184099,train,7,3,9,30.0
4,878,True,61911,train,9,2,13,30.0


- Frequency of banana orders
    - % of orders
    - Every n days on average
- Recency of banana orders
    - n of orders
    - n days

In [68]:
USER = 61911

In [69]:
prior = pd.merge(prior, orders[['order_id', 'user_id']])

In [70]:
prior['bananas'] = prior.product_id == BANANAS

In [71]:
# This user has ordered 196 products
df = prior[prior.user_id==USER]
df

,order_id,product_id,add_to_cart_order,reordered,product_name,aisle_id,department_id,user_id,bananas
2394584,748936,21903,5,1,Organic Baby Spinach,123,4,61911,False
2394585,748936,21137,3,1,Organic Strawberries,24,4,61911,False
2394586,748936,47626,11,1,Large Lemon,24,4,61911,False
2394587,748936,4920,14,0,Seedless Red Grapes,123,4,61911,False
2394588,748936,49683,12,1,Cucumber Kirby,83,4,61911,False
...,...,...,...,...,...,...,...,...,...
10511837,2271842,14778,4,1,Organic Chocolate Chip Chewy Granola Bars,3,19,61911,False
10511838,2271842,38622,6,1,Cheddar Snack Crackers Cheddar Bunnies,78,19,61911,False
10511839,2271842,41834,7,1,Green Chile Enchilada Sauce,30,6,61911,False
10511840,2271842,7414,13,0,Wheat Hamburger Buns,43,3,61911,False


In [72]:
# This person has ordered bananas 6 times
df['bananas'].sum()

6

In [73]:
df[df['bananas']]

,order_id,product_id,add_to_cart_order,reordered,product_name,aisle_id,department_id,user_id,bananas
2759878,1579677,24852,6,0,Banana,24,4,61911,True
3634685,738971,24852,4,1,Banana,24,4,61911,True
3640671,837210,24852,6,1,Banana,24,4,61911,True
8657479,364645,24852,7,1,Banana,24,4,61911,True
9051757,768788,24852,6,1,Banana,24,4,61911,True
10511829,2271842,24852,1,1,Banana,24,4,61911,True


In [74]:
# How many unique orders for this user?
df['order_id'].nunique()

8

In [75]:
df['bananas'].sum() / df['order_id'].nunique()

0.75